In [29]:
import pandas as pd

In [42]:
data = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv')

In [43]:
data.head().T

,0,1,2,3,4
lead_source,paid_ads,social_media,events,paid_ads,referral
industry,NaN,retail,healthcare,retail,education
number_of_courses_viewed,1,1,5,2,3
annual_income,79450.0,46992.0,78796.0,83843.0,85012.0
employment_status,unemployed,employed,unemployed,NaN,self_employed
location,south_america,south_america,australia,australia,europe
interaction_count,4,1,3,1,3
lead_score,0.94,0.8,0.69,0.87,0.62
converted,1,0,1,0,1


In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1334 non-null   object 
 1   industry                  1328 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1281 non-null   float64
 4   employment_status         1362 non-null   object 
 5   location                  1399 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB


In [45]:
categorical = ['lead_source', 'industry', 'employment_status', 'location']
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score', 'converted']

In [46]:
data[categorical] = data[categorical].fillna('NA')
data[numerical] = data[numerical].fillna(0)

In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1462 non-null   object 
 1   industry                  1462 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1462 non-null   float64
 4   employment_status         1462 non-null   object 
 5   location                  1462 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB


In [49]:
data['industry'].mode()

,industry
0,retail


In [51]:
data[numerical].corr()

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879,0.435914
annual_income,0.009770,1.000000,0.027036,0.015610,0.053131
interaction_count,-0.023565,0.027036,1.000000,0.009888,0.374573
lead_score,-0.004879,0.015610,0.009888,1.000000,0.193673
converted,0.435914,0.053131,0.374573,0.193673,1.000000


In [53]:
X = data.drop('converted', axis = 1)
Y = data.converted

In [55]:
from sklearn.model_selection import train_test_split

X_full_train, X_test, y_full_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_full_train, y_full_train, test_size=0.25, random_state=42)

In [56]:
len(X_train), len(X_val), len(X_test)

(876, 293, 293)

In [60]:
from sklearn.metrics import mutual_info_score

for c in categorical:
  print(c)
  info = mutual_info_score(y_train, X_train[c])
  print(round(info, 2))
  print()
  print()

lead_source
0.04


industry
0.01


employment_status
0.01


location
0.0




In [61]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=False)

train_dict = X_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = X_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

In [89]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

pred = model.predict(X_val)
pred = (pred >= 0.5).astype(int)
accuracy = (y_val == pred).mean()
print(round(accuracy, 2))

0.7


In [74]:
from sklearn.model_selection import train_test_split

X_full_train, X_test, y_full_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_full_train, y_full_train, test_size=0.25, random_state=42)

In [75]:
print('without industry')

X_tr_ind = X_train.drop('industry', axis = 1)
X_val_ind = X_val.drop('industry', axis = 1)

dv = DictVectorizer(sparse=False)

train_dict = X_tr_ind.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = X_val_ind.to_dict(orient='records')
X_val = dv.transform(val_dict)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

pred = model.predict(X_val)
pred = (pred >= 0.5).astype(int)
accuracy_ind = (y_val == pred).mean()
print(accuracy - accuracy_ind)

without industry
0.0


In [77]:
from sklearn.model_selection import train_test_split

X_full_train, X_test, y_full_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_full_train, y_full_train, test_size=0.25, random_state=42)

In [78]:
print('without employment_status')

X_tr_ind = X_train.drop('employment_status', axis = 1)
X_val_ind = X_val.drop('employment_status', axis = 1)

dv = DictVectorizer(sparse=False)

train_dict = X_tr_ind.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = X_val_ind.to_dict(orient='records')
X_val = dv.transform(val_dict)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

pred = model.predict(X_val)
pred = (pred >= 0.5).astype(int)
accuracy_ind = (y_val == pred).mean()
print(accuracy - accuracy_ind)

without employment_status
0.0034129692832763903


In [79]:
from sklearn.model_selection import train_test_split

X_full_train, X_test, y_full_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_full_train, y_full_train, test_size=0.25, random_state=42)

In [80]:
print('without lead_score')

X_tr_ind = X_train.drop('lead_score', axis = 1)
X_val_ind = X_val.drop('lead_score', axis = 1)

dv = DictVectorizer(sparse=False)

train_dict = X_tr_ind.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = X_val_ind.to_dict(orient='records')
X_val = dv.transform(val_dict)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

pred = model.predict(X_val)
pred = (pred >= 0.5).astype(int)
accuracy_ind = (y_val == pred).mean()
print(accuracy - accuracy_ind)

without lead_score
-0.0068259385665528916


In [91]:
from sklearn.model_selection import train_test_split

X_full_train, X_test, y_full_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_full_train, y_full_train, test_size=0.25, random_state=42)

from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=False)

train_dict = X_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = X_val.to_dict(orient='records')
X_val = dv.transform(val_dict)
c_s = [0.01, 0.1, 1, 10, 100]

for c in c_s:
  print(c)
  model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
  model.fit(X_train, y_train)

  pred = model.predict(X_val)
  pred = (pred >= 0.5).astype(int)
  accuracy = (y_val == pred).mean()
  accuracy = round(accuracy, 3)
  print(accuracy)
  print()
  print()

0.01
0.7


0.1
0.7


1
0.7


10
0.7


100
0.7


